## 环境要求

- TensorFlow的版本：2.0 + 
- keras
- sklearn
- librosa

In [1]:
# 基本库

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

## 加载深度学习框架

In [ ]:
# 搭建分类模型所需要的库

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical 

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## 加载音频处理库

In [7]:
!pip install librosa --user

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 213 kB 182 kB/s eta 0:00:01
     |████████████████████████████████| 377 kB 185 kB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 186 kB/s eta 0:00:01
     |████████████████████████████████| 56 kB 186 kB/s eta 0:00:01
     |████████████████████████████████| 323 kB 170 kB/s eta 0:00:01
     |████████████████████████████████| 25.3 MB 192 kB/s eta 0:00:01
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23141 sha256=38eef06ec5b63502f8aa9d9db7ad07ed32e23c2ed0752a7ddb2cfac18ead0f83
  Stored in directory: /home/admin/.cache/pip/wheels/ed/45/01/f5cb442bb82226a14111f47d92918c29d5375aa2b68dad31e5
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320718 sha256=b36992552849d586ac6924845b22ab7b0a4f8570e70f6c619c025ff357af4810
  Stored in directory: /home/admin/.cache/pip/wheels/ec/fa/88/6c3015728966b48e44ad4f1d2260e05f5ef5a8b0d80d1098f

In [3]:
# 其他库

import os
import librosa
import librosa.display
import glob 

## 特征提取以及数据集的建立

In [4]:
feature = []
label = []
# 建立类别标签，不同类别对应不同的数字。
label_dict = {'aloe': 0, 'burger': 1, 'cabbage': 2,'candied_fruits':3, 'carrots': 4, 'chips':5,
                  'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream':11,
                  'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon':17,
                  'soup': 18, 'wings': 19}
label_dict_inv = {v:k for k,v in label_dict.items()}

In [5]:
# glob模块可以查找符合特定规则的文件路径名。
parent_dir = './train_sample/'
sub_dir = "aloe"
file_ext = "*.wav"
os.path.join(parent_dir, sub_dir, file_ext)

'./train_sample/aloe/*.wav'

In [6]:
# aloe文件夹下代表了所有食物为aloe的声音文件

glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:5]

['./train_sample/aloe/24EJ22XBZ5.wav',
 './train_sample/aloe/2KZFB8S8ZH.wav',
 './train_sample/aloe/4EDPONVXHL.wav',
 './train_sample/aloe/9634XOIM1L.wav',
 './train_sample/aloe/B3D7HM4M22.wav']

In [7]:
#读取单文件示例
'''
X : 音频的信号值，类型是ndarray
sample_rate : 采样率
'''
X, sample_rate = librosa.load('./train_sample/aloe/24EJ22XBZ5.wav', res_type='kaiser_fast')

In [8]:
X

array([ 0.00217551, -0.01149546, -0.02177233, ..., -0.01755694,
       -0.01340434, -0.01044544], dtype=float32)

In [11]:
len(X)

71661

In [9]:
sample_rate

22050

In [12]:
from tqdm import tqdm
def extract_features(parent_dir, sub_dirs, max_file=10, file_ext="*.wav"):
    c = 0
    label, feature = [], []
    for sub_dir in sub_dirs:
        for fn in tqdm(glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:max_file]): # 遍历数据集的所有文件
            
            label_name = fn.split('/')[-2]
            label.extend([label_dict[label_name]])
            
            # 使用librosa读取wav格式文件
            X, sample_rate = librosa.load(fn, res_type='kaiser_fast')
            
            mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
            
            feature.extend([mels])
            
    return [feature, label]

In [13]:
# 自己更改目录
parent_dir = './train_sample/'
save_dir = "./"

folds = sub_dirs = np.array(['aloe','burger','cabbage','candied_fruits',
                             'carrots','chips','chocolate','drinks','fries',
                            'grapes','gummies','ice-cream','jelly','noodles','pickles',
                            'pizza','ribs','salmon','soup','wings'])

# 获取特征feature以及类别的label
temp = extract_features(parent_dir, sub_dirs, max_file=100)

100%|██████████| 35/35 [00:07<00:00,  5.21it/s]


In [ ]:
temp = np.array(temp)
data = temp.transpose()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [17]:
data[0]

array([array([9.55628872e+00, 8.91002178e+00, 5.43598080e+00, 3.29708529e+00,
       2.37345934e+00, 1.90455341e+00, 1.80879819e+00, 1.68337762e+00,
       3.03310204e+00, 5.18077660e+00, 7.56116772e+00, 7.96967268e+00,
       6.85043240e+00, 3.17706203e+00, 1.45846856e+00, 9.95601535e-01,
       1.01416063e+00, 9.00417864e-01, 7.27246523e-01, 6.79472983e-01,
       8.07720423e-01, 5.92221320e-01, 4.79039162e-01, 4.19901192e-01,
       4.00979280e-01, 3.04575920e-01, 3.08832645e-01, 6.98438048e-01,
       5.78885019e-01, 4.85829115e-01, 4.84552830e-01, 4.75085944e-01,
       4.28770155e-01, 3.27101856e-01, 3.58252019e-01, 3.85662675e-01,
       6.74188733e-01, 1.10357356e+00, 1.87173676e+00, 2.92070651e+00,
       2.26331949e+00, 3.13808155e+00, 3.33568430e+00, 1.90762460e+00,
       1.96067417e+00, 2.06479001e+00, 1.98696399e+00, 1.58567047e+00,
       1.01973081e+00, 7.48169184e-01, 3.92181695e-01, 3.56813818e-01,
       2.80595899e-01, 2.04729810e-01, 1.90995812e-01, 1.99260846e-01,

In [18]:
# 获取特征
X = np.vstack(data[:, 0])

# 获取标签
Y = np.array(data[:, 1])
print('X的特征尺寸是：',X.shape)
print('Y的特征尺寸是：',Y.shape)

X的特征尺寸是： (1000, 128)
Y的特征尺寸是： (1000,)


In [19]:
# 在Keras库中：to_categorical就是将类别向量转换为二进制（只有0和1）的矩阵类型表示
Y = to_categorical(Y)

In [20]:
'''最终数据'''
print(X.shape)
print(Y.shape)

(1000, 128)
(1000, 20)


In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1, stratify=Y)
print('训练集的大小',len(X_train))
print('测试集的大小',len(X_test))

训练集的大小 750
测试集的大小 250


In [22]:
'''
关于输入的选择，您可以使用语谱图输入(2D)或原始语音信号(1D)作为输入。
对于语谱图输入，您必须使用2D CNN，而对于原始语音信号，您可以使用1D CNN。
Mel谱图通常比线性谱图更受欢迎，因为我们的耳朵听到的频率是对数尺度的，而不是线性的。

'''

X_train = X_train.reshape(-1, 16, 8, 1)
X_test = X_test.reshape(-1, 16, 8, 1)

## 建立模型

### 搭建CNN网络

In [23]:
model = Sequential()

# 输入的大小
input_dim = (16, 8, 1)

model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))# 卷积层
model.add(MaxPool2D(pool_size=(2, 2)))# 最大池化
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh")) #卷积层
model.add(MaxPool2D(pool_size=(2, 2))) # 最大池化层
model.add(Dropout(0.1))
model.add(Flatten()) # 展开
model.add(Dense(1024, activation = "tanh"))
model.add(Dense(20, activation = "softmax")) # 输出层：20个units输出20个类的概率

# 编译模型，设置损失函数，优化方法以及评价标准
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [24]:
# 训练模型
model.fit(X_train, Y_train, epochs = 20, batch_size = 15, validation_data = (X_test, Y_test))

Epoch 1/20
50/50 [==============================] - 3s 58ms/step - loss: 2.9994 - accuracy: 0.0887 - val_loss: 2.7905 - val_accuracy: 0.1800
Epoch 2/20
50/50 [==============================] - 3s 59ms/step - loss: 2.5569 - accuracy: 0.2030 - val_loss: 2.5647 - val_accuracy: 0.2160
Epoch 3/20
50/50 [==============================] - 3s 60ms/step - loss: 2.2742 - accuracy: 0.2873 - val_loss: 2.4674 - val_accuracy: 0.2680
Epoch 4/20
50/50 [==============================] - 3s 64ms/step - loss: 1.9957 - accuracy: 0.3937 - val_loss: 2.4836 - val_accuracy: 0.2560
Epoch 5/20
50/50 [==============================] - 3s 66ms/step - loss: 1.8979 - accuracy: 0.4112 - val_loss: 2.4241 - val_accuracy: 0.2880
Epoch 6/20
50/50 [==============================] - 3s 64ms/step - loss: 1.7388 - accuracy: 0.4630 - val_loss: 2.4312 - val_accuracy: 0.3040
Epoch 7/20
50/50 [==============================] - 3s 69ms/step - loss: 1.6102 - accuracy: 0.4995 - val_loss: 2.4741 - val_accuracy: 0.3120
Epoch 8/20
50

### 预测测试集

In [25]:
def extract_features(test_dir, file_ext="*.wav"):
    feature = []
    for fn in tqdm(glob.glob(os.path.join(test_dir, file_ext))[:]): # 遍历数据集的所有文件
        X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
        mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
        feature.extend([mels])
    return feature

In [ ]:
X_test = extract_features('./test_a/')

 91%|█████████▏| 1826/2000 [06:02<00:33,  5.14it/s]

In [ ]:
X_test = np.vstack(X_test)
predictions = model.predict(X_test.reshape(-1, 16, 8, 1))

In [ ]:
preds = np.argmax(predictions, axis = 1)
preds = [label_dict_inv[x] for x in preds]

path = glob.glob('./test_a/*.wav')
result = pd.DataFrame({'name':path, 'label': preds})

result['name'] = result['name'].apply(lambda x: x.split('/')[-1])
result.to_csv('submit.csv',index=None)